<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
本章でやること：固有表現抽出
つまり文章から人名、組織名といったものを抽出する
まずは簡単な方法を学んで、次にBERTを使ったものを学んでいく
'''

'\n本章でやること：固有表現抽出\nつまり文章から人名、組織名といったものを抽出する\nまずは簡単な方法を学んで、次にBERTを使ったものを学んでいく\n'

In [2]:
# まずはディレクトリ移動
!mkdir chap8
%cd ./chap8


/content/chap8


In [3]:
# 次はライブラリ
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# 集合に関する数学的な操作のライブラリかな？
# https://qiita.com/anmint/items/37ca0ded5e1d360b51f3
import itertools

import random
import json
from tqdm import tqdm
import numpy as np

# unicode文字列に対していろいろ操作できそうなライブラリ
# https://docs.python.org/ja/3/library/unicodedata.html
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 31.2 MB/s 
     |████████████████████████████████| 486 kB 60.5 MB/s 
     |████████████████████████████████| 13.4 MB 22.0 MB/s 
     |████████████████████████████████| 582 kB 58.2 MB/s 
     |████████████████████████████████| 120 kB 70.7 MB/s 
     |████████████████████████████████| 880 kB 41.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 419 kB 59.8 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=2cd0f9ef568eebf7693f29ee7d5782bdc8d3a2c9b014f0fbc38d7f599324f4b9
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=32574ef286fee200a311e0e8bdcfb142e246d387f100ad66882f3f6fb23fabe6
  Stored in dire

In [4]:
'''
固有表現抽出とは

人名・組織名などの固有名詞
日付などの時間表現
金額などの数値表現

などを抽出する。そして、それが人名なのかなんなのか、カテゴリを判定する

ちなみに、どんなカテゴリが定義されているかは何パターンかあり、問題によって変える
IREX
拡張固有表現階層
などがある
'''

'\n固有表現抽出とは\n\n人名・組織名などの固有名詞\n日付などの時間表現\n金額などの数値表現\n\nなどを抽出する。そして、それが人名なのかなんなのか、カテゴリを判定する\n\nちなみに、どんなカテゴリが定義されているかは何パターンかあり、問題によって変える\nIREX\n拡張固有表現階層\nなどがある\n'

In [5]:
'''
表記揺れについて

固有表現には全角・半角などの違いで、表記揺れが発生する
よって、これらを正規化することで１つに統一する必要がある

正規化には、unicodedata.normalize('NFKC', text) が用いられる
NFKCは正規化のモード
textは正規化対象の文字列

'''
# 匿名関数lambda
normalize = lambda s: unicodedata.normalize("NFKC", s)

# 全角ABCを半角に正規化
print(f'ＡＢＣ　-> {normalize("ＡＢＣ")}')
# 半角ABCを半角に正規化
print(f'ABC　-> {normalize("ABC")}')

# 全角１２３を半角に正規化
print(f'１２３　-> {normalize("１２３")}')
# 半角ABCを半角に正規化
print(f'123　-> {normalize("123")}')

# 全角カタカナを全角に正規化
print(f'アイウ　-> {normalize("アイウ")}')
# 半角カタカナを全角に正規化
print(f'ｱｲｳ　-> {normalize("ｱｲｳ")}')

ＡＢＣ　-> ABC
ABC　-> ABC
１２３　-> 123
123　-> 123
アイウ　-> アイウ
ｱｲｳ　-> アイウ


In [6]:
'''
固有表現抽出方法１
IO法

簡単なアルゴリズムであるが、一般的にはBIO法の方がよく使われる
    IO法でもまあまあいいが、一部対応できない表現があるためBIOが使われる傾向にある

アルゴリズム
１。トークンが固有表現の一部であれば、トークンにタグ「I-(Type)」をつける.  ex) I - (人名)　など
２。トークンが固有表現の一部でないならば、トークンのタグはOとする
3。O以外の同じタグが連続している部分トークン列を連結して、固有表現とする
    トークナイザがトークン分割の際に「##」を付与している場合があるので、これは除外して結合する

I は Inside, O は Outsideから来ている


問題点
同じタグを持つ固有名詞が複数続いた場合は、全てが連結されて一つの固有表現として抽出される
日米　などは日本とアメリカで別々のはずなのに、1個に合体してしまう

'''

'\n固有表現抽出方法１\nIO法\n\n簡単なアルゴリズムであるが、一般的にはBIO法の方がよく使われる\n    IO法でもまあまあいいが、一部対応できない表現があるためBIOが使われる傾向にある\n\nアルゴリズム\n１。トークンが固有表現の一部であれば、トークンにタグ「I-(Type)」をつける.  ex) I - (人名)\u3000など\n２。トークンが固有表現の一部でないならば、トークンのタグはOとする\n3。O以外の同じタグが連続している部分トークン列を連結して、固有表現とする\n    トークナイザがトークン分割の際に「##」を付与している場合があるので、これは除外して結合する\n\nI は Inside, O は Outsideから来ている\n\n\n問題点\n同じタグを持つ固有名詞が複数続いた場合は、全てが連結されて一つの固有表現として抽出される\n日米\u3000などは日本とアメリカで別々のはずなのに、1個に合体してしまう\n\n'

In [7]:
# 固有表現では普段のトークンにタグを付与するので
# それに対応したトークナイザを定義しないといけない

# そのトークナイザの中では、学習・推論に使用する関数を定義
# ここは最悪あんまり理解してなくてもいいらしい


# BertJapaneseTokenizerを継承：つまり、selfはBertJapaneseTokenizerの関数は使える
class NER_tokenizer(BertJapaneseTokenizer):

    '''
    文章と、それに含まれる固有表現が与えられたときに、符号化とラベル列の作成を行う関数
    この関数の出力は、BERTモデルに入力できる形になっている。（学習時に使用する関数）
    つまり、固有表現に適切なラベルを付与してBERTに入力できるように変換する関数
    これをBERTに入力すると、ラベルがついているため、損失を返してくれる
    これをもとにパラメータを学習する
    '''
    def encode_plus_tagged(self, text, entities, max_length):

        # 手順１：固有表現の前後でtextを分割し、それぞれのラベルをつけておく

        # entities は配列
        # 配列のようそはdict型
        # {'name' : 固有表現, 'span': [文章中の文字列の開始位置index, 終了index+1], 'type' : 固有表現のラベル, 'type_id' : typeに紐づく数字（１：１対応）}
        # sorted :https://note.nkmk.me/python-list-sort-sorted/
        # 単語の文章中での開始位置でソート
        entities = sorted(entities, key=lambda x: x['span'][0])

        # 分割後の文字列を追加する
        splitted = []

        # 次にラベルをつけるのはどこからか？を表す
        position = 0

        '''
        entitiesの中には、文章中に入っている固有表現とそのラベル一覧が入っている（教師データ）
        textを普段通り分割すると、固有表現が途中で分断されてしまって、教師データと一致する固有表現が検出できなくなってしまうことがある
        これを回避するために、以下のfor文では教師データの固有表現を見つけては、textを分割するという対応をとっている
        分割ついでにラベルもちゃんと記録する
        '''
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]

            # 固有表現のラベル
            label = entity['type_id']

            # 固有表現でないものにはラベルOをつける
            splitted.append({'text' : text[position:start], 'label' : 0})

            # 固有表現には対応するラベルをIDで付与
            splitted.append({'text' : text[start:end], 'label' : label})

            # 次にラベルをつけるのはend以降
            position = end

        # 最後余った文字列をラベル0にする
        splitted.append({'text' : text[position:], 'label' : 0})

        # 長さ０の文字列をsplittedから取り除く
        splitted = [ s for s in splitted if s['text']]

        # 手順２：分割されたそれぞれの文字列をトークン化し、ラベルづけする

        # トークンを入れる
        tokens = []
        # ラベルを入れる
        labels = []

        for text_splitted in splitted:
            text = text_splitted['text']
            label = text_splitted['label']

            # テキストをトークン化
            tokens_splitted = self.tokenize(text)
            # 対応するラベルを追加
            labels_splitted = [label] * len(tokens_splitted)

            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        
        # 手順３：符号化を行い、BERTに入力可能にする
        input_ids = self.convert_tokens_to_ids(tokens)

        # 公式ドキュメントが見つからないが
        # prepare_for_model はinput_idsを符号化するらしい
        # 公式これか？？https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.prepare_for_model

        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )


        # 特殊トークン[CLS], [SEP]のラベルを０にする
        labels = [0] + labels[:max_length-2] + [0]

        # [PAD]もラベルを０に
        # [PAD]は文末にあるので、その数だけ最後に追加すればいい
        labels = labels + [0]*(max_length - len(labels))

        encoding['labels'] = labels

        return encoding

    
    '''
    文章を符号化するとともに、各トークンの文章中の位置を特定する関数
    文章中に未知語などが含まれていた場合、先ほどの関数でラベルづけした出力（input_ids やそれに対応するlabel 元の文章は保存されない）からは
    元の文章を復元して、固有表現はこれだ！と特定できない。
    そのため、位置を特定しておく必要がある
    （推論時に使用する関数）
    '''
    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None
    ):
        # 手順１：文章のトークン化

        # トークンを追加していく
        tokens = []
        # トークンに対応する文章中の文字列を追加していく
        tokens_original = []

        # MeCabで単語に分割
        # https://huggingface.co/transformers/v4.11.3/_modules/transformers/models/bert_japanese/tokenization_bert_japanese.html
        # self.word_tokenizerにはBasicTokenizerというクラスのインスタンスが入ってるっぽい
        words = self.word_tokenizer.tokenize(text)
        # print('1')
        # 手順２：単語をサブワード分割して、元のテキストを保存
        for word in words:
            # 単語をサブワードに分割
            # 未知語も分割すれば対応可能になる可能性が上がるっぽいね　　　例）東京タワーが未知語でも、東京とタワーはそれぞれ既知の単語かもしれない
            # https://note.com/npaka/n/nb08941a36c8b#I3MMH
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)

            # 未知語への対応
            # 未知語に遭遇した場合は元のテキストを残しておかないと、全て[UNK]に変換されてしまって区別できなくなる
            if tokens_word[0] == '[UNK]':
                tokens_original.append(word)
            else:
                # 先頭の単語以外は##を除去して保存
                # 未知語対策のifを挟まないと、[UNK]が直で保存される
                tokens_original.extend([
                    token.replace('##', '') for token in tokens_word
                ])
        # print('2')
        # 手順３：各トークンの文章中での位置を保存（空白位置も考慮）
        position = 0
        spans = [] # トークンの位置情報を追加していく

        for token in tokens_original:
            # トークン長
            l = len(token)

            # text（元の入力文章）中の[position..position+l]を切り出して、tokenと一致するか確認
            # while 1 = while true
            while 1:
                # 一致しない場合
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += 1
                    break

        # print('3')
        # 手順４：符号化を行い、BERT入力可能にする
        input_ids = self.convert_tokens_to_ids(tokens)

        # input_idsが既にわかっている場合に使用できる符号化
        # 確かめてはいないが、いつもはBertJapanezeTokenizerのインスタンスを関数として呼び出していたが、それができないため、わざわざこの関数を使用しているのではないのか？と思っている
        # selfに直接引数与えれるのかな・・・
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length' if max_length else False,
            truncation = True if max_length else False
        )

        # [CLS], [SEP], [PAD]の処理
        # 系列長
        sequence_length = len(encoding['input_ids'])

        # [CLS]に対応するダミーspan追加
        # sequence_lengthには[sep][cls]が入っているが、spansの時点ではこれらは入っていないデータを使っているので、数合わせのために-2
        # sequence_lenghtはspansの長さより長い
        spans = [[-1, -1]] + spans[:sequence_length-2]

        # sep, padに対する追加
        spans = spans + [[-1, -1]] * (sequence_length - len(spans))

        # print('4')
        
        # 手順５：必要に応じて出力の型を調整
        if return_tensors == 'pt':
            encoding = { k : torch.tensor([v]) for k, v in encoding.items() }
        
        return encoding, spans


    '''
    「文章」と「タグ列」と「各トークンの文章中の位置」が与えられた時に、文章中に含まれる固有表現に対応する文字列や位置を特定する関数
    文章、ラベル列の予測値、各トークンの位置から固有表現を得る（推論時に使用）
    '''
    def convert_bert_output_to_entities(self, text, labels, spans):
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):
            
            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities 





In [8]:
# 上記のトークナイザーをインスタンス化
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [9]:
# encode_plus_taggedの挙動確認
text = '昨日のみらい事務所との打ち合わせは順調だった'

# ちなみにIO法のOタグは0(zero), I タグは１以上の数字で表される 
entities = [
    {'name' : 'みらい事務所', 'span' : [3, 9], 'type_id' : 1}
]

encoding = tokenizer.encode_plus_tagged(
    text, entities, max_length=20
)
print(encoding)

{'input_ids': [2, 10271, 28486, 5, 546, 10780, 2464, 13, 5, 1878, 2682, 9, 10750, 308, 10, 3, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], 'labels': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [10]:
# encode_plus_untaggedの挙動確認

text = '騰訊の英語名はTencent Holdings Ltdである。'
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
)
print('# encoding')
print(encoding)
print('# spans')
print(spans)

# encoding
{'input_ids': tensor([[    2,     1, 26280,     5,  1543,   125,     9,  6749, 28550,  2953,
         28550, 28566, 21202, 28683, 14050, 12475,    12,    31,     8,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
# spans
[[-1, -1], [0, 1], [1, 2], [2, 3], [3, 5], [5, 6], [6, 7], [7, 9], [9, 10], [10, 12], [12, 13], [13, 14], [15, 18], [18, 19], [19, 23], [24, 27], [27, 28], [28, 30], [30, 31], [-1, -1]]


In [11]:
# convert_bert_output_to_entitiesの挙動確認
# 適当なバートからの出力を置いておく
labels_predicted = [0, 1, 1, 0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0]

entities = tokenizer.convert_bert_output_to_entities(
    text, labels_predicted, spans
)
print(entities)

[{'name': '騰訊', 'span': [0, 2], 'type_id': 1}, {'name': 'Tencent Holdings Ltd', 'span': [7, 27], 'type_id': 1}]


In [12]:
'''
ここからは、BERTも使って固有表現抽出を行っていく
BertForTokenClassificationが使用できる
'''

tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# 固有表現のタイプが３つ　＋　Oタグでlabel数は4
# 1:組織名、２：人名、３：製品名
bert_tc = BertForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels = 4
)

bert_tc = bert_tc.cuda()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the m

In [13]:
'''
推論時の使い方
'''

# データの定義
text = 'AさんはB大学に入学した。'

# 符号化を行い、トークンの文章中の位置も特定する
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors = 'pt'
)

encoding = { k : v.cuda() for k, v in encoding.items() }

# BERTでトークンごとの分類スコアを出力
# スコアが最も高いラベルを予測値とする

with torch.no_grad():
    output = bert_tc(**encoding)
    # scores[バッチサイズ, 系列長, ラベルの数]
    scores = output.logits
    labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

# ラベルを固有表現に変換
entities = tokenizer.convert_bert_output_to_entities(text, labels_predicted, spans)
print(entities)


# ちなみに、チューニングしていないので出力は出鱈目である

[{'name': 'AさんはB大学に', 'span': [0, 8], 'type_id': 1}, {'name': '入学', 'span': [8, 10], 'type_id': 2}, {'name': 'した。', 'span': [10, 13], 'type_id': 1}]


In [14]:
'''
学習する際のコード

'''

# 学習用データの定義
data = [
    {
        'text': 'AさんはB大学に入学した。',
        'entities': [
            {'name': 'A', 'span': [0, 1], 'type_id': 2},
            {'name': 'B大学', 'span': [4, 7], 'type_id': 1}
        ]
    },
    {
        'text': 'CDE株式会社は新製品「E」を販売する。',
        'entities': [
            {'name': 'CDE株式会社', 'span': [0, 7], 'type_id': 1},
            {'name': 'E', 'span': [12, 13], 'type_id': 3}
        ]
    }
]

# 各データを符号化
max_length = 32

# 符号化結果を格納
dataset_for_loader = []

# 符号化
for sample in data:
    text = sample['text']
    entities = sample['entities']

    encoding = tokenizer.encode_plus_tagged(
        text, entities, max_length
    )
    encoding = { k : torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)


# データローダを作成
dataloader = DataLoader(dataset_for_loader, batch_size = len(data))

# ミニバッチを取り出して損失を得る
for batch in dataloader:
    batch = { k : v.cuda() for k, v in batch.items() }
    output = bert_tc(**batch)
    loss = output.loss


In [15]:
# データセット使って学習しましょうや
'''
データの詳細
https://github.com/stockmarketeam/ner-wikipedia-dataset

データは以下のような形をしている
{
    'curid' : 'wikipediaのID',
    'entities' : [
        { 'name' : '長岡半太郎', 'span' : [0, 5], 'type' : '人名' },
        { 'name' : '半太郎', 'span' : [34, 37], 'type' : '人名' },
        { 'name' : '長三郎', 'span' : [46, 49], 'type' : '人名' }
    ],
    'text' : '長岡半太郎とは同郷の先輩後輩関係でかねてから親交があったのみならず、半太郎は妻や幼子の医療を長三郎に全面的に頼んでいた。'
}
'''
!git clone --branch v2.0 https://github.com/stockmarkteam/ner-wikipedia-dataset 

Cloning into 'ner-wikipedia-dataset'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 9), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (32/32), done.
Note: checking out 'f7ed83626d90e5a79f1af99775e4b8c6cba15295'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [16]:
'''
データを読み込み、
文字列の正規化を行い
固有表現のカテゴリを1~8のタイプのIDに変更する
それから学習データ・訓練データなどに分割する

'''

# データのロード
dataset = json.load(open('ner-wikipedia-dataset/ner.json', 'r'))

# 固有表現のタイプとIDを対応づける辞書
type_id_dict = {
    "人名": 1,
    "法人名": 2,
    "政治的組織名": 3,
    "その他の組織名": 4,
    "地名": 5,
    "施設名": 6,
    "製品名": 7,
    "イベント名": 8
}


# カテゴリーをラベルに変更
# 文字列の正規化
for sample in dataset:
    # カテゴリをIDへ
    for e in sample['entities']:
        e['type_id'] = type_id_dict[e['type']]
        del e['type']

    # 文字列の正規化
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])


# データの分割
# 学習：検証：テスト = 6 : 2 : 2

random.shuffle(dataset)
n = len(dataset)
n_train = int(n * 0.6)
n_val = int(n*0.2)

dataset_train = dataset[:n_train]
dataset_val = dataset[n_train : n_train + n_val]
dataset_test = dataset[n_train + n_val : ]

In [17]:
'''
ファインチューニングを行うための準備

データローダの作成
'''

def create_dataset(tokenizer, dataset, max_length):
    # まずはデータローダに入力できるようにデータを符号化しよう関数
    dataset_for_loader = []

    for sample in dataset:
        text = sample['text']
        entities = sample['entities']

        encoding = tokenizer.encode_plus_tagged(
            text, entities, max_length=max_length
        )

        encoding = { k : torch.tensor(v) for k, v in encoding.items() }

        dataset_for_loader.append(encoding)
    return dataset_for_loader


# 上の関数を使って、学習・検証用データローダを作る
max_length = 128

dataset_train_for_loader = create_dataset (
    tokenizer, dataset_train, max_length
)

dataset_val_for_loader = create_dataset (
    tokenizer, dataset_val, max_length
)

dataloader_train = DataLoader(
    dataset_train_for_loader, batch_size=32, shuffle=True
)

dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

In [18]:
'''
ファインチューニング
PyTorch Lightning のクラスを定義して
チェックポイントを書き（=モデルを保存する条件を指定する）
学習メソッドを回すぜ

'''

# PyTorch Lightningのモデル
class BertForTokenClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()
        self.bert_tc = BertForTokenClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    def training_step(self, batch, batch_idx):
        output = self.bert_tc(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_tc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

trainer = pl.Trainer(
    gpus=1,
    # max_epochs=5,
    max_epochs=1,
    callbacks=[checkpoint]
)

# ファインチューニング
model = BertForTokenClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)
trainer.fit(model, dataloader_train, dataloader_val)
best_model_path = checkpoint.best_model_path

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [19]:
'''
ここまでで学習が完了した
次はテストデータを用いて予測を行う
'''

# BERTで固有表現抽出を行うための関数を定義しよう
def predict(text, tokenizer, bert_tc):
    # まずはテキストをBERTに入力可能な状態にする
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors = 'pt'
    )

    encoding = { k: v.cuda() for k, v in encoding.items() }

    # BERTに入力して出力を得る
    with torch.no_grad():
        output = bert_tc(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

    # print(labels_predicted)
    # ラベル列を固有表現に変換
    entities = tokenizer.convert_bert_output_to_entities(
        text, labels_predicted, spans
    )
    # print(entities)

    return entities


# ここから本体
# トークナイザーをロード
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# ファインチューニングしたモデルをロードしてGPUにのせる
model = BertForTokenClassification_pl.load_from_checkpoint(
    best_model_path
)
bert_tc = model.bert_tc.cuda()


# 固有表現抽出
# 以下では１データずつ処理しているが、これはわかりやすいコードにするためである
# バッチ化して処理を行った方が処理時間は短い

# 正解の固有表現の置き場所
entities_list = []
# 抽出された固有表現をおく場所
entities_predicted_list = []

for sample in tqdm(dataset_test):
    text=sample['text']
    entities_predicted = predict(text, tokenizer, bert_tc) # BERTで予測
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

In [20]:
print("# 正解")
print(entities_list[0])
print("# 抽出")
print(entities_predicted_list[0])

print(predict('伊藤家には、海外の収集品の他にも、ヤン・フレデリク・フェイルケによる富士山の墨絵「富嶽図」も残されている。', tokenizer, bert_tc))
print(predict('AさんはB大学に入学した。', tokenizer, bert_tc))

# 正解
[{'name': '出島地区', 'span': [6, 10], 'type_id': 5}, {'name': '横浜新都市センター', 'span': [22, 31], 'type_id': 2}]
# 抽出
[{'name': '出', 'span': [6, 7], 'type_id': 5}, {'name': '島', 'span': [7, 8], 'type_id': 6}, {'name': '横浜', 'span': [22, 24], 'type_id': 5}, {'name': '新', 'span': [24, 25], 'type_id': 6}, {'name': '都市センター', 'span': [25, 31], 'type_id': 2}]
[{'name': 'ヤン・フレデリク・フェイルケ', 'span': [17, 31], 'type_id': 1}, {'name': '富嶽', 'span': [41, 43], 'type_id': 1}]
[]


In [21]:
def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    """
    正解と予測を比較し、モデルの固有表現抽出の性能を評価する。
    type_idがNoneのときは、全ての固有表現のタイプに対して評価する。
    type_idが整数を指定すると、その固有表現のタイプのIDに対して評価を行う。
    """
    num_entities = 0 # 固有表現(正解)の個数
    num_predictions = 0 # BERTにより予測された固有表現の個数
    num_correct = 0 # BERTにより予測のうち正解であった固有表現の数

    # それぞれの文章で予測と正解を比較。
    # 予測は文章中の位置とタイプIDが一致すれば正解とみなす。
    for entities, entities_predicted \
        in zip(entities_list, entities_predicted_list):

    # 関数の引数でtype_idを指定した場合には、それと一致する固有表現だけを評価する
        if type_id:
            entities = [ e for e in entities if e['type_id'] == type_id ]
            entities_predicted = [ 
                e for e in entities_predicted if e['type_id'] == type_id
            ]

        # 各entitiy e の初期位置と最終位置、type_idの組みを求める  
        get_span_type = lambda e: (e['span'][0], e['span'][1], e['type_id'])
        set_entities = set( get_span_type(e) for e in entities )
        set_entities_predicted = \
            set( get_span_type(e) for e in entities_predicted )

        num_entities += len(entities)
        num_predictions += len(entities_predicted)
        num_correct += len( set_entities & set_entities_predicted )

    # 指標を計算
    precision = num_correct/num_predictions # 適合率
    recall = num_correct/num_entities # 再現率
    f_value = 2*precision*recall/(precision+recall) # F値

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    return result


In [22]:
print( evaluate_model(entities_list, entities_predicted_list) )

{'num_entities': 2727, 'num_predictions': 2591, 'num_correct': 658, 'precision': 0.2539560015438055, 'recall': 0.2412907957462413, 'f_value': 0.2474614516735615}


In [23]:
'''
BIO法
トークンが
・　固有表現の先頭：B-Typeタグ（BeginningのB）
・　固有表現の先頭以外：I-Typeタグ
・　固有表現以外：Oタグ


こうすることで、同じタグを持つ固有表現が二つ以上並ぶ（日米など）場合にも、
これらを一つの固有表現としてしまうのではなく
分割して抽出できる

Bタグm個
Iタグm個
O タグ１個

この時、ラベルはBタグに1, 2, ......, mが与えられ
対応するIタグには順番に m+1, m+2, ....., 2mが与えられる
Oタグは0



BIO法の難しいところ
Iタグは同じカテゴリのBタグ、同じカテゴリのIタグの後にしか現れないというルールを守らせること

なぜ難しいか？
BERTは各トークンに対する予測値を独立に予測しているため、
単純にBERT出力のスコアが高いラベルを採用すると、ルールを守れないから

解決策
ルールを守れないラベルの組み合わせに対するスコアにペナルティを与える
    たとえば、スコアから-10000するなど
これはViterbiアルゴリズムを使うと達成できる
    が、この本では詳しく解説してくれないらしい
'''

'\nBIO法\nトークンが\n・\u3000固有表現の先頭：B-Typeタグ（BeginningのB）\n・\u3000固有表現の先頭以外：I-Typeタグ\n・\u3000固有表現以外：Oタグ\n\n\nこうすることで、同じタグを持つ固有表現が二つ以上並ぶ（日米など）場合にも、\nこれらを一つの固有表現としてしまうのではなく\n分割して抽出できる\n\nBタグm個\nIタグm個\nO タグ１個\n\nこの時、ラベルはBタグに1, 2, ......, mが与えられ\n対応するIタグには順番に m+1, m+2, ....., 2mが与えられる\nOタグは0\n\n\n\nBIO法の難しいところ\nIタグは同じカテゴリのBタグ、同じカテゴリのIタグの後にしか現れないというルールを守らせること\n\nなぜ難しいか？\nBERTは各トークンに対する予測値を独立に予測しているため、\n単純にBERT出力のスコアが高いラベルを採用すると、ルールを守れないから\n\n解決策\nルールを守れないラベルの組み合わせに対するスコアにペナルティを与える\n    たとえば、スコアから-10000するなど\nこれはViterbiアルゴリズムを使うと達成できる\n    が、この本では詳しく解説してくれないらしい\n'

In [49]:
# まずはいつも通り、トークナイザーを設定しましょう
class NER_tokenizer_BIO(BertJapaneseTokenizer):

    # 初期化時に、固有表現のカテゴリ数num_entitiy_typeを受け入れるようにしよう
    # *args, **kwargsについて
    # https://note.nkmk.me/python-args-kwargs-usage/
    # 前者は引数をタプルとして受け取り、後者は辞書として受け取る
    def __init__(self, *args, **kwargs):
        self.num_entity_type = kwargs.pop('num_entity_type')
        super().__init__(*args, **kwargs)
    

    def encode_plus_tagged(self, text, entities, max_length):
        # 学習時に使う関数
        # 文章とその中に含まれる固有表現を受け取り、
        # 文章を符号化してラベル（BIO）の列を生成する

        # 固有表現の前後で文章を分割

        # 分割後の文字列を追加する
        splitted = []
        # 今どこまで分割したんだっけ
        position = 0

        for entity in entities:
            # entities は配列
            # 配列のようそはdict型
            # {'name' : 固有表現, 'span': [文章中の文字列の開始位置index, 終了index+1], 'type' : 固有表現のラベル, 'type_id' : typeに紐づく数字（１：１対応）}
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']

            splitted.append({'text' : text[position:start], 'label' : 0})
            splitted.append({'text' : text[start:end], 'label' : label})
            position = end
        splitted.append({'text' : text[position:], 'label' : 0})

        # empty文字列の排除
        splitted = [s for s in splitted if s['text']]

        # 分割されたそれぞれの文章をトークン化し、ラベルをつける。
        tokens = [] # トークンを追加していく
        labels = [] # ラベルを追加していく
        for s in splitted:
            tokens_splitted = self.tokenize(s['text'])
            label = s['label']
            if label > 0: # 固有表現
                # まずトークン全てにI-タグを付与
                labels_splitted =  \
                    [ label + self.num_entity_type ] * len(tokens_splitted)
                # 先頭のトークンをB-タグにする
                labels_splitted[0] = label
            else: # それ以外
                labels_splitted =  [0] * len(tokens_splitted)
            
            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length',
            truncation=True
        ) 

        # ラベルに特殊トークンを追加
        # [SEP], [CSL]に対応する０
        labels = [0] + labels[:max_length-2] + [0] 
        # [PAD]
        labels = labels + [0] * (max_length - len(labels))
        encoding['labels'] = labels

        return encoding



    '''
    推論時
    BERTに入力可能なように文章を符号化し、元の文章中での位置を特定する関数
    ''' 

    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None 
    ):
        # まずは文章をトークン化
        tokens = []
        tokens_original = []

        words = self.word_tokenizer.tokenize(text) # MeCabで形態素解析(この時点では[UNK]は発生していないっぽい)

        for word in words:
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)

            # 未知語だった場合
            if tokens_word == '[UNK]':
                tokens_original.append(word)
            else:
            # 未知語ではなかった場合
                tokens_original.extend(
                    [token.replace('##', '') for token in tokens_word]
                )
        
        # さて、次は位置を特定しようじゃないか
        position = 0
        spans = []
        for token in tokens_original:
            if token == text[position : position + len(token)]:
                spans.append([position, position + len(token)])
                position += 1
                break
            else:
                position += 1
        
        # 符号化する
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids,
            max_length = max_length,
            padding = 'max_length' if max_length else False,
            truncation = True if max_length else False
        )

        # spansに特殊トークンに対応する部分を付け加える
        # [CSL], [SEP]
        spans = [[-1, -1]] + spans[:len(encoding['input_ids'])-2] + [[-1, -1]]
        # [PAD]
        spans = spans + [[-1, -1]] * (len(encoding['input_ids']) - len(spans))

        # 必要に応じてtorch.Tensorに変換
        if return_tensors == 'pt':
            encoding = { k : torch.tensor([v]) for k, v in encoding.items() }

        return encoding, spans


    @staticmethod
    def Viterbi(scores_bert, num_entity_type, penalty=10000):
        """
        Viterbiアルゴリズムで最適解を求める。
        """

        # ルールに従わないラベルの並びに対してpenaltyを設定する
        m = 2*num_entity_type + 1
        penalty_matrix = np.zeros([m, m])
        for i in range(m):
            for j in range(1+num_entity_type, m):
                if not ( (i == j) or (i+num_entity_type == j) ): 
                    penalty_matrix[i,j] = penalty
        
        # まずは、１トークン目に対してpenaltyを減算
        path = [ [i] for i in range(m) ]
        scores_path = scores_bert[0] - penalty_matrix[0,:]
        scores_bert = scores_bert[1:]

        # ２トークン目以降は、自身以前のトークンのスコアも参照しながらスコアを更新する
        for scores in scores_bert:
            assert len(scores) == 2*num_entity_type + 1
            score_matrix = np.array(scores_path).reshape(-1,1) \
                + np.array(scores).reshape(1,-1) \
                - penalty_matrix
            scores_path = score_matrix.max(axis=0)
            argmax = score_matrix.argmax(axis=0)
            path_new = []
            for i, idx in enumerate(argmax):
                path_new.append( path[idx] + [i] )
            path = path_new

        labels_optimal = path[np.argmax(scores_path)]
        return labels_optimal


    def convert_bert_output_to_entities(self, text, scores, spans):
        # バートの出力から、予想される固有表現を抽出しましょう

        """
        文章、分類スコア、各トークンの位置から固有表現を得る。
        分類スコアはサイズが（系列長、ラベル数）の2次元配列
        """
        assert len(spans) == len(scores)
        num_entity_type = self.num_entity_type

        # 特殊トークンに対するscores, spansを取り除く
        scores = [score for score, span in zip(scores, spans) if span[0] != 1]
        spans = [span for span in spans if span[0] != 1]

        # Voterbiのアルゴリズムでラベルの予測値を求める
        labels = self.Viterbi(scores, num_entity_type)

        # 各トークンに関して、分類スコア最大のラベルを求める
        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):

            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]
            if label != 0: # 固有表現であれば
                if 1 <= label <= num_entity_type:
                     # ラベルが`B-`ならば、新しいentityを追加
                    entity = {
                        "name": text[start:end],
                        "span": [start, end],
                        "type_id": label
                    }
                    entities.append(entity)
                else:
                    # ラベルが`I-`ならば、直近のentityを更新
                    entity['span'][1] = end 
                    entity['name'] = text[entity['span'][0]:entity['span'][1]]
                
        return entities

In [50]:
# さて、ファインチューニングに移りますよ
'''
まずはトークナイザをロードして、データローダを作成します
'''

# tokenizer BIOは、initをいじったので、ラベル数を指定しないといけない
tokenizer = NER_tokenizer_BIO.from_pretrained(MODEL_NAME, num_entity_type=8)

# データセットの作成
max_length = 128
dataset_train_for_loader = create_dataset(
    tokenizer, dataset_train, max_length
)
dataset_val_for_loader = create_dataset(
    tokenizer, dataset_val, max_length
)

# データローダ作成
dataloader_train = DataLoader(dataset_train_for_loader, batch_size=32, shuffle=True)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer_BIO'.


In [51]:
# BertForTokenClassification_plはIO法の時のをそのまま転用できる

# チェックポイント生成
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model_BIO/'
)


trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks=[checkpoint]
)

# PyTorch Lightningモデルロード
num_entity_type = 8
num_labels = 2 * num_entity_type + 1
model = BertForTokenClassification_pl(
    MODEL_NAME, num_labels = num_labels, lr=1e-5
)

# ファインチューニング
trainer.fit(model, dataloader_train, dataloader_val)
best_model_path = checkpoint.best_model_path


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [54]:
# 性能評価

# 学んだモデルをロード
model = BertForTokenClassification_pl.load_from_checkpoint(
    best_model_path
)

bert_tc = model.bert_tc.cuda()


# 正解の固有表現を追加
entities_list = []
# 予測した固有表現を追加
entities_predicted_list = []

for sample in tqdm(dataset_test):
    text = sample['text']
    

    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(text, return_tensors = 'pt')
    encoding = { k : v.cuda() for k, v in encoding.items() }
    with torch.no_grad():
        output = bert_tc(**encoding)
        scores = output.logits
        scores = scores[0].cpu().numpy().tolist()
    
    # 分類スコアをを固有表現に変換
    entities_predicted = tokenizer.convert_bert_output_to_entities(
        text, scores, spans
    )

    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

print(evaluate_model(entities_list, entities_predicted_list))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the m

{'num_entities': 2727, 'num_predictions': 2766, 'num_correct': 43, 'precision': 0.01554591467823572, 'recall': 0.015768243491015767, 'f_value': 0.015656289823411614}
